SpeechBrain - Use Sepformer to separate overlapped voices and denoise them

- pip install speechbrain
- Took over 60 mins to do a 5 min audio.
- Need to run this on "Run as Admin" - then activate env - then start code and run.

In [7]:
from speechbrain.inference.separation import SepformerSeparation as separator
import torchaudio
import sounddevice as sd

In [2]:
model = separator.from_hparams(source="speechbrain/sepformer-wsj02mix")

In [5]:
# for custom file, change path
est_sources = model.separate_file(path='multiple_voice.wav')

Resampling the audio from 48000 Hz to 8000 Hz


In [6]:
print(est_sources)

tensor([[[ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         ...,
         [ 3.8546e-05, -3.9968e-05],
         [ 1.7644e-05, -3.4395e-05],
         [ 9.0104e-07, -2.6598e-05]]])


In [8]:
est_sources[:, :, 0]

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.8546e-05, 1.7644e-05,
         9.0104e-07]])

In [14]:
def play_wav(waveform, sample_rate, playback_duration=30):
    # Define the duration to play (in seconds)
    num_samples_to_play =  int(playback_duration * sample_rate)
    waveform_to_play = waveform[:, :num_samples_to_play]
    
    # Convert tensor to numpy array
    waveform_np = waveform_to_play.numpy()
    
    # Check if the waveform has only one channel, if yes, expand it to two channels
    if waveform_np.ndim == 1:
        waveform_np = waveform_np.reshape(1, -1)

    # Play audio
    sd.play(waveform_np.T, sample_rate)  # Transpose to match the expected shape (channels, samples)
    sd.wait()

In [15]:
num_of_distinct_voices = est_sources.shape[2]

In [21]:
for i in range(0, num_of_distinct_voices):
    print("Speaker {} Voice Playing!!!".format(i))
    play_wav(waveform = est_sources[:, :, i].detach().cpu(), sample_rate = 8000, playback_duration=5)

    ## Save the files
    torchaudio.save("multiple_voice_{}_speaker.wav".format(i), est_sources[:, :, i].detach().cpu(), 8000)

Speaker 0 Voice Playing!!!
Speaker 1 Voice Playing!!!


In [ ]:
### The results files are generally full length 5 min files where mostly one voice and muffled voices from the other person (Toned down)! 

### There were 3 people in multiple_voices.wav where the second audio had both the voices sometimes - especially when the voices were somewhat similar. It's not good quality results - Not sure how to improve.